In [2]:
import json
import pandas as pd

# Ajustar para mostrar más columnas
pd.set_option('display.max_columns', None)

# Ajustar para mostrar más filas
pd.set_option('display.max_rows', None)

# Ajustar para que el contenido de las columnas no se trunque
pd.set_option('display.max_colwidth', None)

# Mostrar más ancho para una vista más clara
pd.set_option('display.width', 1000)

## Tablas que No Tienen IDs
### Product
### Customer_address
### Time
### Country
### Supplier 
### Weather
### Currency_conversion
### Order_details

In [3]:
df = pd.read_json('../data/raw/data_0_0_0.json', lines=True, orient='records')
orders_df = pd.json_normalize(df['order'], max_level=0)

details_df = pd.json_normalize(df['order'], 'details', ['order_id'])



display(details_df.head())

,brand_product,cantidad,customer_address,customer_id,customer_name,customer_nation,customer_phone,customer_region,descuento,fecha_entrega,fecha_envio,flag_devolución,impuesto,name_prodcuct,numero_item,precio,status,supplier_name,order_id
0,Brand#25,33,"DN,o6cL5oexUKeoePA49RnNBkUciRlYNO",130837,Customer#000130837,UNITED STATES,34-387-803-5958,AMERICA,0.08,1998-08-15,1998-07-23,N,0.08,azure misty papaya magenta olive,1,38716.26,O,Supplier#000008231,3004327
1,Brand#33,4,"DN,o6cL5oexUKeoePA49RnNBkUciRlYNO",130837,Customer#000130837,UNITED STATES,34-387-803-5958,AMERICA,0.00,1998-05-23,1998-05-22,N,0.02,lemon blue medium goldenrod dim,2,6798.96,O,Supplier#000002261,3004327
2,Brand#33,35,"DN,o6cL5oexUKeoePA49RnNBkUciRlYNO",130837,Customer#000130837,UNITED STATES,34-387-803-5958,AMERICA,0.03,1998-07-21,1998-06-29,N,0.06,orchid drab peach lace lawn,3,69579.65,O,Supplier#000006016,3004327
3,Brand#41,17,ze9zPRMacUEaAyF,127691,Customer#000127691,UNITED KINGDOM,33-453-155-9463,EUROPE,0.07,1998-08-07,1998-08-03,N,0.07,saddle black misty drab mint,1,26869.52,O,Supplier#000001587,3005729
4,Brand#42,45,ze9zPRMacUEaAyF,127691,Customer#000127691,UNITED KINGDOM,33-453-155-9463,EUROPE,0.09,1998-09-29,1998-09-08,N,0.00,turquoise forest sienna red rose,2,54189.90,O,Supplier#000002733,3005729


In [52]:
print(orders_df.columns)


Index(['details', 'order_date', 'order_id', 'order_status', 'order_total'], dtype='object')


In [25]:
# Extraer datos relevantes para Product
product_df = pd.json_normalize(df['order'], 'details', ['order_id'], sep='_')

# Seleccionar columnas necesarias
product_df = product_df[['brand_product', 'name_prodcuct', 'precio', 'status']]

# Eliminar duplicados para obtener productos únicos
product_df = product_df.drop_duplicates(subset=['brand_product', 'name_prodcuct', 'precio', 'status'])

# Generar IDs autoincrementales
product_df['id'] = pd.RangeIndex(start=1, stop=len(product_df) + 1)

print(product_df.tail())


      brand_product                       name_prodcuct    precio status     id
67170      Brand#11     royal honeydew wheat slate navy  30373.59      O  66950
67171      Brand#54   firebrick red salmon tomato ivory  23676.66      O  66951
67172      Brand#42  pink medium turquoise misty salmon  29902.34      O  66952
67173      Brand#23  thistle pink yellow cornsilk ghost  22442.42      O  66953
67174      Brand#24        tan beige moccasin drab cyan  16783.20      O  66954


In [27]:
# Extraer datos relevantes para Customer_address
customer_address_df = pd.json_normalize(df['order'], 'details', ['order_id'], sep='_')

# Seleccionar columnas necesarias y eliminar duplicados
customer_address_df = customer_address_df[['customer_address', 'customer_region', 'customer_nation']].drop_duplicates()

# Generar IDs autoincrementales
customer_address_df['id'] = pd.RangeIndex(start=1, stop=len(customer_address_df) + 1)

print(customer_address_df.head())


                           customer_address customer_region customer_nation  id
0         DN,o6cL5oexUKeoePA49RnNBkUciRlYNO         AMERICA   UNITED STATES   1
3                           ze9zPRMacUEaAyF          EUROPE  UNITED KINGDOM   2
10  FmXvUhyxyOw1GkK,kkJt5BhsJK3 g3JGcINMxHt          EUROPE         ROMANIA   3
16                             nB IT5jlB72f          EUROPE          FRANCE   4
22            NMOORDJlyvu44TNn7MPw4E2e1EoGn            ASIA           CHINA   5


In [49]:
# Extraer países únicos de customer_nation
unique_nations = customer_address_df[['customer_nation']].drop_duplicates().reset_index(drop=True)
unique_nations.columns = ['name']  # Asegúrate de que el nombre de la columna coincida con la tabla `country`

# Generar IDs autoincrementales
unique_nations['id'] = pd.RangeIndex(start=1, stop=len(unique_nations) + 1)

# Imprimir para ver el resultado
print(unique_nations)

# Aquí iría el código para insertar unique_nations en la tabla `country` en la base de datos PostgreSQL


              name  id
0    UNITED STATES   1
1   UNITED KINGDOM   2
2          ROMANIA   3
3           FRANCE   4
4            CHINA   5
5        INDONESIA   6
6           CANADA   7
7             IRAN   8
8         ETHIOPIA   9
9          ALGERIA  10
10          JORDAN  11
11      MOZAMBIQUE  12
12         VIETNAM  13
13           INDIA  14
14           JAPAN  15
15          RUSSIA  16
16          BRAZIL  17
17           EGYPT  18
18            IRAQ  19
19    SAUDI ARABIA  20
20         GERMANY  21
21         MOROCCO  22
22            PERU  23
23           KENYA  24
24       ARGENTINA  25


In [50]:
# Crear un DataFrame de mapeo para países
country_mapping = unique_nations[['name', 'id']]
country_mapping.columns = ['customer_nation', 'country_id']

# Unir customer_address_df con country_mapping para obtener country_id
customer_address_df = pd.merge(customer_address_df, country_mapping, left_on='customer_nation', right_on='customer_nation', how='left')

# Seleccionar columnas necesarias y eliminar duplicados
customer_address_df = customer_address_df[['customer_address', 'customer_region', 'country_id']].drop_duplicates()

# Generar IDs autoincrementales
customer_address_df['id'] = pd.RangeIndex(start=1, stop=len(customer_address_df) + 1)

# Imprimir para ver el resultado
print(customer_address_df.head())

# Aquí iría el código para insertar customer_address_df en la tabla `customer_address` en la base de datos PostgreSQL


                          customer_address customer_region  country_id  id
0        DN,o6cL5oexUKeoePA49RnNBkUciRlYNO         AMERICA           1   1
1                          ze9zPRMacUEaAyF          EUROPE           2   2
2  FmXvUhyxyOw1GkK,kkJt5BhsJK3 g3JGcINMxHt          EUROPE           3   3
3                             nB IT5jlB72f          EUROPE           4   4
4            NMOORDJlyvu44TNn7MPw4E2e1EoGn            ASIA           5   5


In [28]:
# Extraer fechas únicas
time_df = pd.DataFrame({
    'date': pd.to_datetime(df['order'].apply(lambda x: x['order_date'])),
    'month': pd.to_datetime(df['order'].apply(lambda x: x['order_date'])).dt.month,
    'year': pd.to_datetime(df['order'].apply(lambda x: x['order_date'])).dt.year
})

# Eliminar duplicados
time_df = time_df.drop_duplicates()

# Generar IDs autoincrementales
time_df['id'] = pd.RangeIndex(start=1, stop=len(time_df) + 1)

print(time_df.head)


        date  month  year  id
0 1998-05-09      5  1998   1
1 1998-05-14      5  1998   2
2 1998-07-15      7  1998   3
3 1998-03-05      3  1998   4
4 1998-06-23      6  1998   5


In [29]:
# Extraer datos relevantes para Supplier
supplier_df = pd.json_normalize(df['order'], 'details', ['order_id'], sep='_')

# Seleccionar columnas necesarias y eliminar duplicados
supplier_df = supplier_df[['supplier_name']].drop_duplicates()

# Generar IDs autoincrementales
supplier_df['id'] = pd.RangeIndex(start=1, stop=len(supplier_df) + 1)

print(supplier_df.head())


        supplier_name  id
0  Supplier#000008231   1
1  Supplier#000002261   2
2  Supplier#000006016   3
3  Supplier#000001587   4
4  Supplier#000002733   5


In [30]:
# Suponiendo que obtendrás estos datos de una API más adelante
weather_df = pd.DataFrame({
    'date': pd.to_datetime([]),  # Placeholder
    'country_id': [],  # Placeholder
    'description': [],  # Placeholder
    'temperature': []  # Placeholder
})

# Generar IDs autoincrementales
weather_df['id'] = pd.RangeIndex(start=1, stop=len(weather_df) + 1)

print(weather_df)


Empty DataFrame
Columns: [date, country_id, description, temperature, id]
Index: []


In [31]:
# Suponiendo que obtendrás estos datos de una API más adelante
currency_conversion_df = pd.DataFrame({
    'base_currency': [],  # Placeholder
    'target_currency': [],  # Placeholder
    'rate': [],  # Placeholder
    'creation_date': []  # Placeholder
})

# Generar IDs autoincrementales
currency_conversion_df['id'] = pd.RangeIndex(start=1, stop=len(currency_conversion_df) + 1)

print(currency_conversion_df)


Empty DataFrame
Columns: [base_currency, target_currency, rate, creation_date, id]
Index: []


In [48]:
# Extraer datos relevantes para Order_details
order_details_df = pd.json_normalize(df['order'], 'details', ['order_id'], sep='_')

# Seleccionar columnas necesarias
order_details_df = order_details_df[['order_id', 'cantidad', 'descuento', 'flag_devolución', 'impuesto', 'numero_item', 'supplier_name']]

# Eliminar duplicados para obtener detalles únicos
order_details_df = order_details_df.drop_duplicates()

# Mapeo de supplier_name a supplier_id
supplier_mapping = supplier_df.set_index('supplier_name')['id'].to_dict()
order_details_df['supplier_id'] = order_details_df['supplier_name'].map(supplier_mapping)

## Se corrobora que el supplier_name corresponda al supplier_name y luego se borra el supplier_name de la orden_detial:
# Crear un DataFrame de mapeo para proveedores
# supplier_mapping = supplier_df[['supplier_name', 'id']]
# supplier_mapping.columns = ['supplier_name', 'supplier_id']

# # Unir order_details_df con supplier_mapping para verificar el mapeo
# merged_df = pd.merge(order_details_df[['supplier_name', 'supplier_id']], supplier_mapping, on='supplier_name', how='left', suffixes=('', '_mapped'))

# # Verificar si los IDs generados son los correctos
# discrepancies = merged_df[merged_df['supplier_id'] != merged_df['supplier_id_mapped']]
# print(discrepancies)

# Seleccionar las columnas finales para Order_details
order_details_df = order_details_df[['order_id', 'cantidad', 'descuento', 'flag_devolución', 'impuesto', 'numero_item', 'supplier_id']]

# Generar IDs autoincrementales
order_details_df['id'] = pd.RangeIndex(start=1, stop=len(order_details_df) + 1)

print(order_details_df.head())


  order_id  cantidad  descuento flag_devolución  impuesto  numero_item  supplier_id  id
0  3004327        33       0.08               N      0.08            1            1   1
1  3004327         4       0.00               N      0.02            2            2   2
2  3004327        35       0.03               N      0.06            3            3   3
3  3005729        17       0.07               N      0.07            1            4   4
4  3005729        45       0.09               N      0.00            2            5   5


  order_id  cantidad  descuento flag_devolución  impuesto  numero_item  supplier_id  id
0  3004327        33       0.08               N      0.08            1            1   1
1  3004327         4       0.00               N      0.02            2            2   2
2  3004327        35       0.03               N      0.06            3            3   3
3  3005729        17       0.07               N      0.07            1            4   4
4  3005729        45       0.09               N      0.00            2            5   5


Empty DataFrame
Columns: [supplier_name, supplier_id, supplier_id_mapped]
Index: []
